# Lesson 4: Persistence and Streaming

In [1]:
import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var]=getpass.getpass(f"{var}: ")

_set_env("TAVILY_API_KEY")

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
tool = TavilySearchResults(max_results=2)

In [4]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

需要安装pip install langgraph-checkpoint-sqlite

使用SqliteSaver会出现'_GeneratorContextManager' object has no attribute 'get_next_version'错误。

In [5]:
"""
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")
"""

from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [6]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [7]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(
    model_name='qwen2',
    #model_name='deepseek-r1:32b',
    openai_api_base="http://127.0.0.1:11434/v1",
    openai_api_key="EMPTY",
    streaming=True
)

abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [8]:
messages = [HumanMessage(content="What is the weather in sf?")]

In [9]:
thread = {"configurable": {"thread_id": "1"}}

In [10]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_xi6k7uhk', 'function': {'arguments': '{"query":"weather in sf"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'qwen2', 'system_fingerprint': 'fp_ollama'}, id='run-c695e7c9-1e05-4726-b897-bc5468d1c509-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': 'call_xi6k7uhk', 'type': 'tool_call'}])]
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': 'call_xi6k7uhk', 'type': 'tool_call'}
Back to the model!
[ToolMessage(content='[{\'title\': \'Weather in san francisco\', \'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.775, \'lon\': -122.4183, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1741479046, \'localt

In [11]:
messages = [HumanMessage(content="What about in la?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_h2x6t26j', 'function': {'arguments': '{"query":"weather in la"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'qwen2', 'system_fingerprint': 'fp_ollama'}, id='run-463d7dd1-6c6c-4cee-9878-30e72d3076b9-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in la'}, 'id': 'call_h2x6t26j', 'type': 'tool_call'}])]}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in la'}, 'id': 'call_h2x6t26j', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'title\': \'Weather in los angeles, california\', \'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Los Angeles\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 34.0522, \'lon\': -118.2428, \'tz_id\': \'America/Los_Angeles\', \'loc

In [12]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='Based on the information available, San Francisco has a temperature around 12.3°C (54.1°F) while Los Angeles has a temperature at about 17.2°C (63.0°F). Therefore, Los Angeles is warmer than San Francisco.\n\nSource: https://www.weatherapi.com/', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'qwen2', 'system_fingerprint': 'fp_ollama'}, id='run-3f37e96b-5f90-43c1-afb1-5faba8589b4c-0')]}


In [13]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='To answer your question accurately, I need more context. Are you asking about the temperature of two different places or comparing two substances based on their heat content? Could you please provide more details?', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'qwen2', 'system_fingerprint': 'fp_ollama'}, id='run-4332c066-49f1-4d46-9ff7-3e1e58359d3d-0')]}


## Streaming tokens

使用AsyncSqliteSaver会出现'_AsyncGeneratorContextManager' object has no attribute 'get_next_version'错误

In [14]:
"""
#from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver

memory = AsyncSqliteSaver.from_conn_string(":memory:")
"""

from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [15]:
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [16]:
messages = [HumanMessage(content="What is the weather in SF?")]
thread = {"configurable": {"thread_id": "4"}}
async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in SF'}, 'id': 'call_gljnr3he', 'type': 'tool_call'}
Back to the model!
